# Data Cleaning for San Juan and North Cascades Data Sets

In [1]:
import pandas as pd
import calendar

tbl = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/tbl_Plant_Collections.txt", sep=None, engine="python", encoding="latin1")
noca2002 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA 2002.txt", sep=None, engine="python", encoding="latin1")
noca2003 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA 2003.txt", sep=None, engine="python", encoding="latin1")
noca2004 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA 2004.txt", sep=None, engine="python", encoding="latin1")
noca2005 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA 2005.txt", sep=None, engine="python", encoding="latin1")
noca2006 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA_2006_GIS.txt", sep=None, engine="python", encoding="latin1")
noca2007 = pd.read_csv("/Users/carterwebb/Desktop/plants/raw_data/no_ca/NOCA 2007.txt", sep=None, engine="python", encoding="latin1")

## San Juans

### Reduce Size To a Minimal State

In [2]:
tbl2 = tbl[[
    "Family",
    "Scientific Name",
    "Common Name",
    "Coll Date",
    "County",
    "State",
    "UTM Z/E/N",
    "Elevation M",
    "UTM_Y",
    "UTM_X",
]]

### Split Scientific Name Up for Genus and Species Columns

In [3]:
tbl2[["Genus", "Species"]] = tbl2["Scientific Name"].str.split(n=1, expand=True); 
tbl2[["Species", "Authority"]] = tbl2["Species"].str.split(n=1, expand=True)

/var/folders/7s/t5l3pg4j2j54njys9zrdzdxr0000gn/T/ipykernel_31894/1242335323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl2[["Genus", "Species"]] = tbl2["Scientific Name"].str.split(n=1, expand=True);
/var/folders/7s/t5l3pg4j2j54njys9zrdzdxr0000gn/T/ipykernel_31894/1242335323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl2[["Genus", "Species"]] = tbl2["Scientific Name"].str.split(n=1, expand=True);
/var/folders/7s/t5l3pg4j2j54njys9zrdzdxr0000gn/T/ipykernel_31894/1242335323.py:2: SettingWit

### Snake Case Naming

In [4]:
tbl2 = tbl2.rename(columns={
    "Scientific Name": "scientific_name",
    "Family": "family",
    "Genus": "genus",
    "Species": "species",
    "Authority": "authority",
    "Common Name": "common_name",
    "Coll Date": "coll_date",
    "County": "county",
    "State": "state",
    "UTM Z/E/N": "utm_zone",
    "Elevation M": "elev_m",
    "UTM_Y": "utm_y",
    "UTM_X": "utm_x"
})

### Normalize Date Time (coll_date)

In [5]:

tbl2["coll_date"] = pd.to_datetime(
    tbl2["coll_date"],
    format="%m/%d/%Y %H:%M:%S",   # matches "5/4/2006 0:00:00"
    errors="coerce"
)

# Drop Time 
tbl2["coll_date"] = tbl2["coll_date"].dt.normalize()

### Standardize the Column Order for Future 

In [6]:
cols = ["family", "genus", "species",
        "utm_zone", "utm_x", "utm_y", "elev_m",
        "coll_date"]


sanjuan = tbl2.reindex(columns=cols).copy()

### Save in Case

In [7]:
sanjuan.to_csv("SJ_Simplified_2001_2007.csv", index=False, encoding="utf-8-sig")

## North Casecades Data Set

### Snake Case the Columns We are Thinking of Keeping

In [9]:
rename_map = {
    "Full Scientific Name": "scientific_name",
    "Scientific Name": "scientific_name",
    "Family": "family",
    "Genus": "genus",
    "Species": "species",
    "species": "species",
    "Species author": "authority",
    "Author": "authority",
    "County": "county",
    "State": "state",
    "Elevation (m)": "elev_m",
    "Elevation (meters)": "elev_m",
    "UTM Zone": "utm_zone",
    "UTM Z/E/N": "utm_zone",
    "UTM Easting": "utm_x",
    "Easting": "utm_x",
    "UTM Northing": "utm_y",
    "Northing": "utm_y",
    "Lat Deg": "lat_deg",
    "Degrees Lat": "lat_deg",
    "Lat Min": "lat_min",
    "Minutes Lat": "lat_min",
    "Lat Sec": "lat_sec",
    "Seconds Lat": "lat_sec",
    "N or S": "ns",
    "Long Deg": "long_deg",
    "Degrees Long": "long_deg",
    "Long Min": "long_min",
    "Minutes Long": "long_min",
    "Lon Sec": "lon_sec",
    "Seconds Long": "lon_sec",
    "W or E": "we",
    "Dec Latitude": "latitude",
    "Dec Longitude": "longitude",
    "Collection Day": "coll_day",
    "Collection Month": "coll_month",
    "Collection Year": "coll_year",
}

noca2002 = noca2002.rename(columns=rename_map)
noca2003 = noca2003.rename(columns=rename_map)
noca2004 = noca2004.rename(columns=rename_map)
noca2005 = noca2005.rename(columns=rename_map)
noca2006 = noca2006.rename(columns=rename_map)
noca2007 = noca2007.rename(columns=rename_map)

### Standardize Date Time Objects for all Different Syntaxes 


AI Assist

In [11]:
def build_coll_date(df, day_col="coll_day", month_col="coll_month", year_col="coll_year",
                    out_col="coll_date", add_flags=True):
    """
    Create a proper datetime column from coll_day/month/year.
    - Handles month names ("Aug", "August") and numbers (8).
    - Fills missing month/day with 1 so dates still parse.
    - Leaves rows with missing year as NaT.
    - Optionally adds boolean flags for imputed month/day.
    """
    df = df.copy()

    # --- Month name/abbr -> number (case-insensitive) ---
    month_map = {m.lower(): i for i, m in enumerate(calendar.month_name) if m}
    month_map.update({m.lower(): i for i, m in enumerate(calendar.month_abbr) if m})  # Jan, Feb, ...

    # normalize month to string, strip, lower, then map; if not a name, try numeric
    m_raw = df[month_col].astype(str).str.strip().str.lower()
    m_num = m_raw.map(month_map)
    m_num = pd.to_numeric(m_num, errors="coerce")  # keep mapped ints; others become NaN
    m_num = m_num.fillna(pd.to_numeric(df[month_col], errors="coerce"))  # try numeric fallback

    # day/year to numeric
    d_num = pd.to_numeric(df[day_col], errors="coerce")
    y_num = pd.to_numeric(df[year_col], errors="coerce")

    # flags for imputation
    month_imputed = m_num.isna()
    day_imputed = d_num.isna()

     # fill missing month/day with 1 so we can parse; keep year as-is (missing year -> NaT)
    m_num = m_num.fillna(1).astype("Int64")
    d_num = d_num.fillna(1).astype("Int64")

    # build datetime; rows with NaN year will become NaT
    df[out_col] = pd.to_datetime(
        dict(year=y_num, month=m_num, day=d_num),
        errors="coerce"
    )

    if add_flags:
        df[out_col + "_month_imputed"] = month_imputed
        df[out_col + "_day_imputed"] = day_imputed

    return df

# ---- Apply to your NOCA frames (after your renaming to snake_case) ----
nc02 = build_coll_date(noca2002)
nc03 = build_coll_date(noca2003)
nc04 = build_coll_date(noca2004)
nc05 = build_coll_date(noca2005)
nc06 = build_coll_date(noca2006)
nc07 = build_coll_date(noca2007)

# (Optional) if tbl2 has a string "coll_date" already, normalize to datetime too:
# tbl2["coll_date"] = pd.to_datetime(tbl2["coll_date"], errors="coerce")

### Standardize the Columns for Future Combination, Matches San Juans. 

In [13]:
cols = ["family", "genus", "species",
        "utm_zone", "utm_x", "utm_y", "elev_m",
        "coll_date"]


def keep(df, cols):
    return df.reindex(columns=cols).copy()


nc02 = keep(nc02, cols)
nc03 = keep(nc03, cols)
nc04 = keep(nc04, cols)
nc05 = keep(nc05, cols)
nc06 = keep(nc06, cols)
nc07 = keep(nc07, cols)



noca = pd.concat([nc02, nc03, nc04, nc05, nc06, nc07],
                       ignore_index=True)

### Save In Case

In [ ]:
noca.to_csv("NOCA_2002_2007_simplified.csv", index=False, encoding="utf-8-sig")

# Prepare for Final Concat

In [14]:
noca.insert(0, "parkname", "noca")

In [15]:
sanjuan.insert(0, "parkname", "sanjuan")

Merging Via concat or join??

In [ ]:
#merged = pd.merge(noca, sanjuan, how='outer', on='parkname')

In [16]:
merged = pd.concat([noca, sanjuan], axis=0, ignore_index=True)

### Save

In [ ]:
merged.to_csv("SJ_NOCA_2001_2007.csv", index=False, encoding="utf-8-sig")